In [ ]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark()

## Homework ##

**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


In [ ]:
shippers = spark.read.csv('/class/datasets/northwind/TSV/shippers', header=False, sep = '\t')
#shippers.show()
print(shippers)
shippers.createOrReplaceTempView('shippers')
print(shippers.collect())
shippers = spark.sql('select cast(_c0 as int) as ShipperID, _c1 as ShipperName, _c2 as Phone from shippers')
shippers.createOrReplaceTempView('shippers')
print(shippers)

def isEven(x):
    return x % 2 == 0

from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark.udf.register('isEven', isEven, BooleanType())

spark.sql('select * FROM Shippers WHERE isEven(ShipperID)').show()


In [ ]:
o = spark.read.json('Orders_LineItems.json')
o.createOrReplaceTempView('Orders')
sql = '''select OrderId, CustomerID, OrderDate, l.ProductID, l.UnitPrice, l.Quantity, l.Discount
FROM Orders LATERAL VIEW EXPLODE(LineItems) EXPLODED_TABLE AS l
'''
o2 = spark.sql(sql)
o2.show()
o2.createOrReplaceTempView('FlatOrders')
spark.sql('select avg(UnitPrice) from FlatOrders').show()
